In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization

# Dataset Example

In [2]:
BASE_DIR = os.path.abspath(os.path.join("cats_and_dogs", "training_set"))
CATS_DIR = os.path.join(BASE_DIR, "cats")
DOGS_DIR = os.path.join(BASE_DIR, "dogs")

BATCH_SIZE = 16
IMAGE_SIZE = (128, 128)

In [3]:
cat_images = [os.path.join(CATS_DIR, f) for f in os.listdir(CATS_DIR)]
dog_images = [os.path.join(DOGS_DIR, f) for f in os.listdir(DOGS_DIR)]

In [4]:
all_images = cat_images + dog_images

In [5]:
len(all_images)

8000

In [6]:
def preprocess_images(filename):
    file = tf.io.read_file(filename)
    
    img = tf.image.decode_jpeg(file)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = img / 255.0
    
    label = tf.strings.split(filename, sep = "\\")[-2] == "cats"
    label = tf.cast(label, tf.int32)
    
    return img, label

In [7]:
dataset = tf.data.Dataset.from_tensor_slices(all_images) \
    .shuffle(len(all_images)) \
    .map(preprocess_images) \
    .batch(BATCH_SIZE) \
    .prefetch(tf.data.AUTOTUNE) \
    .repeat()

In [8]:
for x in dataset:
    print(x)
    break

(<tf.Tensor: shape=(16, 128, 128, 3), dtype=float32, numpy=
array([[[[0.5019651 , 0.5019651 , 0.45490626],
         [0.51990944, 0.51990944, 0.4885369 ],
         [0.49566483, 0.49174327, 0.476057  ],
         ...,
         [0.76852477, 0.7253875 , 0.64695615],
         [0.7870404 , 0.74390316, 0.6654718 ],
         [0.78456336, 0.7414261 , 0.66299474]],

        [[0.5041973 , 0.5041973 , 0.45713848],
         [0.52429223, 0.52429223, 0.49291968],
         [0.4923237 , 0.48840213, 0.47271585],
         ...,
         [0.774983  , 0.73184574, 0.65341437],
         [0.7873437 , 0.7442064 , 0.66577506],
         [0.78937364, 0.7462364 , 0.667805  ]],

        [[0.5000153 , 0.5000153 , 0.4529565 ],
         [0.5053426 , 0.5053426 , 0.47397006],
         [0.48648298, 0.4825614 , 0.46687514],
         ...,
         [0.7633964 , 0.72025913, 0.64182776],
         [0.7695176 , 0.72638035, 0.647949  ],
         [0.7678723 , 0.724735  , 0.64630365]],

        ...,

        [[0.48693943, 0.48301786

In [9]:
model = Sequential([
    Input((128, 128, 3)),
    Conv2D(filters = 64, kernel_size = (5, 5), padding = "same", activation = "relu"),
    Conv2D(filters = 64, kernel_size = (5, 5), padding = "same", activation = "relu"),
    MaxPool2D(),
    BatchNormalization(),
    Conv2D(filters = 32, kernel_size = (3, 3), padding = "same", activation = "relu"),
    Conv2D(filters = 32, kernel_size = (3, 3), padding = "same", activation = "relu"),
    MaxPool2D(),
    BatchNormalization(),
    Conv2D(filters = 16, kernel_size = (3, 3), padding = "same", activation = "relu"),
    Conv2D(filters = 16, kernel_size = (3, 3), padding = "same", activation = "relu"),
    MaxPool2D(),
    Flatten(),
    Dense(64, activation = "relu"),
    Dropout(0.2),
    Dense(32, activation = "relu"),
    Dropout(0.1),
    Dense(1, activation = "sigmoid")
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      4864      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      102464    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        18464     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0

In [11]:
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:
model.fit(dataset, steps_per_epoch = 500, epochs = 4)

Epoch 1/4
